<a href="https://colab.research.google.com/github/nadyadtm/BERT-Implementation-in-Chunking/blob/main/BERT_implementation_in_Chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT implementation in Chunking


## Mount Google Drive

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Package

In [2]:
#install package transformes dan lakukan import
!pip install transformers==3
import transformers
from transformers import DistilBertModel, DistilBertTokenizer, AdamW, get_linear_schedule_with_warmup

#import package pytorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader,TensorDataset
from tqdm import tqdm, trange


#import package sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

#import numpy, pandas
import numpy as np
import pandas as pd

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from collections import defaultdict
from textwrap import wrap

#import os
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

     |████████████████████████████████| 757kB 18.7MB/s 
     |████████████████████████████████| 890kB 57.6MB/s 
     |████████████████████████████████| 3.0MB 57.0MB/s 
     |████████████████████████████████| 1.2MB 49.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=96929e72c340e1123f79bc5ac563ed728beb8d1601b07bde922530bfc42c6b6d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##Set GPU
Sebelum memulai pelatihan, diperlukan untuk set GPU untuk menjalankan Pytorch

In [3]:
#mengecek apakah terdapat GPU pada komputer
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
#jika tidak ada maka gunakan CPU untuk menjalankan program
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


## Load File txt

In [4]:
DATA_DIR = 'drive/My Drive/SMT 2/NLP/Tugas 3/Chunking/{}.txt'

def get_data(file):
    with open(file, 'r', encoding='latin1') as fp:
        content = fp.readlines()
    data, sent = [], []
    for line in content:
        if not line.strip():
            if sent: data.append(sent)
            sent = []
        else:
            word, pos, tag = line.strip().split()
            # tag = tag.split('-')[0]
            sent.append((word, pos, tag))
    return data

In [5]:
train_data = get_data(DATA_DIR.format('train'))
test_data = get_data(DATA_DIR.format('test'))

Kemudian, mengambil sentence dan labelnya. Di kasus ini kita mengabaikan POSnya

In [6]:
sentences = [[word[0] for word in sentence] for sentence in train_data]
sentences_test = [[word[0] for word in sentence] for sentence in test_data]

In [7]:
labels = [[s[2] for s in sentence] for sentence in train_data]
labels_test = [[s[2] for s in sentence] for sentence in test_data]

Berikut ini adalah tag yang ada pada dataset tersebut

In [10]:
data1 = pd.read_csv("drive/My Drive/SMT 2/NLP/Tugas 3/Chunking/train.txt", 
                  sep=' ', 
                  names=["Words", "POS", "Tag"])
data2 = pd.read_csv("drive/My Drive/SMT 2/NLP/Tugas 3/Chunking/test.txt", 
                  sep=' ', 
                  names=["Words", "POS", "Tag"])
data = pd.concat([data1,data2])
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}
tag2idx

{'B-ADJP': 2,
 'B-ADVP': 19,
 'B-CONJP': 16,
 'B-INTJ': 17,
 'B-LST': 11,
 'B-NP': 21,
 'B-PP': 20,
 'B-PRT': 1,
 'B-SBAR': 0,
 'B-UCP': 8,
 'B-VP': 22,
 'I-ADJP': 9,
 'I-ADVP': 4,
 'I-CONJP': 6,
 'I-INTJ': 15,
 'I-LST': 3,
 'I-NP': 14,
 'I-PP': 13,
 'I-PRT': 7,
 'I-SBAR': 18,
 'I-UCP': 12,
 'I-VP': 5,
 'O': 10,
 'PAD': 23}

## Preprocessing
Inisiasi Tokenizer, max len, dan batch size di awal. Pada kasus ini, model yang digunakan DistilBert

In [11]:
PRE_TRAINED_MODEL_NAME = 'distilbert-base-cased'
MAX_LEN = 30
bs = 32

tokenizer = DistilBertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=False)

Fungsi tersebut untuk melakukan tokenisasi per kata

In [15]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [16]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]


In [17]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

Kemudian berikan padding pada kalimat tersebut

In [18]:
from keras.preprocessing.sequence import pad_sequences
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")


In [19]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


Lalu mendapatkan attention mask

In [20]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

## Train Val split dan Data Loader
Melakukan pembagian data dan load datanya

In [21]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [22]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [23]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_dataloader = DataLoader(train_data, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_dataloader = DataLoader(valid_data, batch_size=bs)

## BERT Model
Inisiasi bert model

In [24]:
from transformers import DistilBertForTokenClassification

model = DistilBertForTokenClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Insert Model ke GPU

In [25]:
model.cuda()

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

Set optimizer, epoch, dan scheduler

In [33]:
epochs = 10

#set optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * epochs

#set schedule
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

## Model Function
Berikut ini fungsi training dan fungsi evaluasi

### Fungsi train_epoch

In [27]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  # Put the model into training mode.
  model.train()
  # Reset the total loss for this epoch.
  total_loss = 0

  # Reset the validation loss for this epoch.
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions , true_labels = [], []

  # Training loop
  for step, batch in enumerate(data_loader):
      # add batch to gpu
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      # Always clear any previously calculated gradients before performing a backward pass.
      model.zero_grad()
      # forward pass
      # This will return the loss (rather than the model output)
      # because we have provided the `labels`.
      outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

      # Move logits and labels to CPU
      logits = outputs[1].detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.extend(label_ids)

      # get the loss
      loss = outputs[0]
      # Perform a backward pass to calculate the gradients.
      loss.backward()
      # track train loss
      total_loss += loss.item()
      # Clip the norm of the gradient
      # This is to help prevent the "exploding gradients" problem.
      torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=1)
      # update parameters
      optimizer.step()
      # Update the learning rate.
      scheduler.step()

    # Calculate the average loss over the training data.
  avg_train_loss = total_loss / len(data_loader)
  pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
  valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
  return accuracy_score(pred_tags, valid_tags),avg_train_loss

### Fungsi eval_model

In [28]:
def eval_model(model, data_loader, loss_fn, device, n_examples):

  # Put the model into evaluation mode
  model.eval()
  # Reset the validation loss for this epoch.
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions , true_labels = [], []
  
  for batch in data_loader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch

      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
          # Forward pass, calculate logit predictions.
          # This will return the logits rather than the loss because we have not provided labels.
          outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

      # Move logits and labels to CPU
      logits = outputs[1].detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      # Calculate the accuracy for this batch of test sentences.
      eval_loss += outputs[0].mean().item()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.extend(label_ids)

  eval_loss = eval_loss / len(data_loader)
  pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
  valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
  return accuracy_score(pred_tags, valid_tags),eval_loss

## Proses

In [29]:
from sklearn.metrics import accuracy_score
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for epoch in range(epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    #memanggil fungsi train epoch
    train_acc, train_loss = train_epoch(
      model,
      train_dataloader,    
      loss_fn, 
      optimizer, 
      device, 
      scheduler, 
      len(train_data)
    )

    #mencetak train loss dan accuracy
    print(f'Train loss {train_loss} accuracy {train_acc}')


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    val_acc, val_loss = eval_model(
      model,
      valid_dataloader,    
      loss_fn, 
      device, 
      len(valid_data)
    )
    
    #mencetak val loss dan accuracy
    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()


Epoch 1/10
----------
Train loss 0.3131703030436285 accuracy 0.9067194255147651
Val   loss 0.11487518091286932 accuracy 0.9690081680280047

Epoch 2/10
----------
Train loss 0.08379571719302072 accuracy 0.9769651100712873
Val   loss 0.10872599801846913 accuracy 0.9711085180863477

Epoch 3/10
----------
Train loss 0.05066881858788076 accuracy 0.9862083765139382
Val   loss 0.11123650560953788 accuracy 0.9713885647607935

Epoch 4/10
----------
Train loss 0.033988714092485016 accuracy 0.9909608600529682
Val   loss 0.11501048279127904 accuracy 0.9729288214702451

Epoch 5/10
----------
Train loss 0.024016724021676632 accuracy 0.9935045902291451
Val   loss 0.11690898652055434 accuracy 0.9732088681446908

Epoch 6/10
----------
Train loss 0.018030951543348946 accuracy 0.9951585382449308
Val   loss 0.12185985994126115 accuracy 0.9732555425904318

Epoch 7/10
----------
Train loss 0.014567284473395418 accuracy 0.9961268305959446
Val   loss 0.12559700783874309 accuracy 0.9732555425904318

Epoch 8/10

## Pengujian dengan data test

In [30]:
tokenized_texts_and_labels_test = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences_test, labels_test)
]

tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
labels_test = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]

input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test]

ts_inputs = torch.tensor(input_ids_test)
ts_tags = torch.tensor(tags_test)
ts_masks = torch.tensor(attention_masks_test)

test_data = TensorDataset(ts_inputs, ts_masks, ts_tags)
test_dataloader = DataLoader(test_data, batch_size=bs)

In [31]:
test_acc, test_loss = eval_model(
      model,
      test_dataloader,    
      loss_fn, 
      device, 
      len(test_data)
)

In [32]:
test_acc

0.9718374118622978